### RAPTOR Recursive Abstractive Processing for Tree Organized Retrieval ###

In [ ]:
import os

In [ ]:
from config import set_environment
set_environment()

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import Settings
from llama_index.core.response.notebook_utils import display_response, display_source_node

from llama_index.core import SimpleDirectoryReader
#from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor, KeywordNodePostprocessor
from llama_index.core import get_response_synthesizer

from llama_index.packs.raptor import RaptorPack
from llama_index.vector_stores.chroma import ChromaVectorStore
import chromadb

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large",dimensions=512,)
Settings.llm = OpenAI(temperature=0, model="gpt-4")

In [ ]:
from llama_index.llms.cohere import Cohere
from llama_index.core import ServiceContext
from llama_index.embeddings.cohere import CohereEmbedding

Settings.llm = Cohere(api_key=os.environ["COHERE_API_KEY"], model="command-r")
Settings.embed_model = CohereEmbedding(
    cohere_api_key=os.environ["COHERE_API_KEY"],
    model_name="embed-english-v3.0",
    input_type="search_query",
)

In [ ]:
reader = SimpleDirectoryReader("data")
documents = reader.load_data()

In [ ]:
client = chromadb.PersistentClient(path="./chroma/orcl_utd_spd")
collection = client.get_or_create_collection("orcl_utd_spd")

vector_store = ChromaVectorStore(chroma_collection=collection)

raptor_pack = RaptorPack(
    documents,
    embed_model=Settings.embed_model,
    llm=Settings.llm,  # used for generating summaries
    vector_store=vector_store,  # used for storage
    similarity_top_k=3,  # top k for each layer, or overall top-k for collapsed
    mode="collapsed",  # sets default mode
    transformations=[
        SentenceSplitter(chunk_size=1024, chunk_overlap=100)
    ],  # transformations applied for ingestion
)

In [ ]:
nodes = raptor_pack.run("what vision coverage do you have?", mode="collapsed")
print(len(nodes))
print(nodes[0].text)

In [ ]:
from llama_index.packs.raptor import RaptorRetriever

retriever = RaptorRetriever(
    [],
    embed_model = Settings.embed_model,
    llm=Settings.llm,  # used for generating summaries
    vector_store=vector_store,  # used for storage
    similarity_top_k=3,  # top k for each layer, or overall top-k for collapsed
    mode="tree_traversal",  # sets default mode
)

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine_minimal = RetrieverQueryEngine.from_args(
    retriever, llm=Settings.llm
)

In [ ]:
query_engine_minimal = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=node_postprocessors
)
query_engine_refine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "refine"),
    node_postprocessors=node_postprocessors
)

query_engine_compact = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "compact"),
    node_postprocessors=node_postprocessors
)

query_engine_tree_summarize = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "tree_summarize"),
    node_postprocessors=node_postprocessors
)

query_engine_simple_summarize = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "simple_summarize"),
    node_postprocessors=node_postprocessors
)

query_engine_accumulate = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "accumulate"),
    node_postprocessors=node_postprocessors
)

query_engine_compact_accumulate = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=get_response_synthesizer(response_mode = "compact_accumulate"),
    node_postprocessors=node_postprocessors
)


In [ ]:
def generate_answer(value, response_mode):
    if response_mode == "minimal":
        return query_engine_minimal.query(value)
    elif response_mode == "refine":
        return query_engine_refine.query(value)
    elif response_mode == "compact":
        return query_engine_compact.query(value)
    elif response_mode == "tree_summarize":
        return query_engine_tree_summarize.query(value)
    elif response_mode == "simple_summarize":
        return query_engine_simple_summarize.query(value)
    elif response_mode == "accumulate":
        return query_engine_accumulate.query(value)
    elif response_mode == "compact_accumulate":
        return query_engine_compact_accumulate.query(value)

In [ ]:
import pandas as pd

In [ ]:
questions_path = 'questions/ORCL_UTD_SPD_Questions.xlsx' 
df = pd.read_excel(questions_path, sheet_name='final')


In [ ]:
df['generated_answer_minimal'] = df['question'].apply(generate_answer, response_mode = "minimal")

In [ ]:
df['generated_answer_minimal'] = df['question'].apply(generate_answer, response_mode = "minimal")
df['generated_answer_refine'] = df['question'].apply(generate_answer, response_mode = "refine")
df['generated_answer_compact'] = df['question'].apply(generate_answer, response_mode = "compact")
df['generated_answer_tree_summarize'] = df['question'].apply(generate_answer, response_mode = "tree_summarize")
df['generated_answer_simple_summarize'] = df['question'].apply(generate_answer, response_mode = "simple_summarize")
df['generated_answer_accumulate'] = df['question'].apply(generate_answer, response_mode = "accumulate")
df['generated_answer_tree_compact_accumulate'] = df['question'].apply(generate_answer, response_mode = "compact_accumulate")

In [ ]:
output_file_path = 'result/output.xlsx'  
df.to_excel(output_file_path, index=False)

In [ ]:
retrievals = retriever.retrieve(query)
for n in retrievals:
    display_source_node(n, source_length=source_length)